In [1]:
import numpy as np
import pandas as pd

# Question a

"constructor_results" dataset has points and constructor Id and race Id, so we need "races" to select the races from 1981 to 2020 and "constructors" to get the name of constructors

In [17]:
constructor_results = pd.read_csv("data_raw/constructor_results.csv")
races = pd.read_csv("data_raw/races.csv")
constructors = pd.read_csv("data_raw/constructors.csv")

In [18]:
merged = pd.merge(
    constructor_results,
    races[['raceId', 'year']],
    on="raceId",   
    how="left"           
)
print(merged)

merged2 = pd.merge(
    merged,
    constructors[["constructorId", "name"]],
    on="constructorId",   
    how="left"           
)
print(merged2)

       constructorResultsId  raceId  constructorId  points status  year
0                         1      18              1    14.0     \N  2008
1                         2      18              2     8.0     \N  2008
2                         3      18              3     9.0     \N  2008
3                         4      18              4     5.0     \N  2008
4                         5      18              5     2.0     \N  2008
...                     ...     ...            ...     ...    ...   ...
12165                 16665    1096            117     5.0     \N  2022
12166                 16666    1096            213     0.0     \N  2022
12167                 16667    1096             51     0.0     \N  2022
12168                 16668    1096              3     0.0     \N  2022
12169                 16669    1096            210     0.0     \N  2022

[12170 rows x 6 columns]
       constructorResultsId  raceId  constructorId  points status  year  \
0                         1      18

In [19]:
subset = merged2.query("1981 <= year <= 2020")

agg_points = (subset.groupby("name")
                     .agg(total_points=("points", "sum")))


agg_points = agg_points.sort_values(by="total_points",ascending=False )
print(agg_points)

            total_points
name                    
Ferrari           7374.0
Mercedes          5685.0
McLaren           5229.5
Red Bull          5043.5
Williams          3355.0
...                  ...
RAM                  0.0
Spyker MF1           0.0
Spirit               0.0
Simtek               0.0
Virgin               0.0

[67 rows x 1 columns]


In [21]:
avg_points = agg_points["total_points"].mean()
print(avg_points)

532.2388059701492


TOP3: Ferrari           7374.0  

Mercedes          5685.0

McLaren           5229.5

average points 532.2388

The top3 points are much higher than average

# Question b

same as part a

In [22]:
subset2 = merged2.query("2001 <= year <= 2020")

agg_points2 = (subset2.groupby("name")
                     .agg(total_points=("points", "sum")))


agg_points2 = agg_points2.sort_values(by="total_points",ascending=False )
print(agg_points2)

                total_points
name                        
Ferrari               5862.0
Mercedes              5685.0
Red Bull              5043.5
McLaren               3284.0
Williams              1535.5
Renault               1465.0
Force India           1098.0
Lotus F1               706.0
Toro Rosso             500.0
Sauber                 467.0
BMW Sauber             308.0
Racing Point           283.0
Toyota                 278.5
BAR                    207.0
Haas F1 Team           200.0
Brawn                  172.0
AlphaTauri             107.0
Honda                  106.0
Alfa Romeo              65.0
Jordan                  58.0
Jaguar                  45.0
Benetton                10.0
Minardi                 10.0
Prost                    4.0
Super Aguri              4.0
Arrows                   3.0
Marussia                 2.0
Manor Marussia           1.0
Spyker                   1.0
Caterham                 0.0
HRT                      0.0
MF1                      0.0
Lotus         

In [23]:
avg_points2 = agg_points2["total_points"].mean()
print(avg_points2)

786.0142857142857


TOP3

Ferrari               5862.0

Mercedes              5685.0

Red Bull              5043.5

average     786.0

The top3 points are still much higher than average


# Question c

It is easy to find out that Ferrari and Mercedes are #1 and #2, but the third place changed from McLaren in the 1st period  to Red Bull in the 2nd period

# Question d

to get driver id, we need to use "results" dataset

In [24]:
results = pd.read_csv("data_raw/results.csv")

In [33]:
ferrari_id = constructors.query("name == 'Ferrari'")["constructorId"].iloc[0]
print(ferrari_id)



6


In [37]:
ferrari_results = (results[results["constructorId"] == ferrari_id]
                   .merge(races[["raceId", "year"]], on="raceId", how="left")
                   .query("1981 <= year <= 2020"))

ferrari_driver_ids = ferrari_results["driverId"].unique()

print(ferrari_driver_ids)
len(ferrari_driver_ids)

[  8  13  30  22  56  63  77  55  78 122 117  81  95  69  21 105 140 163
 175 202 203 207   4  20 844]


25

so there are 25 drivers

 # question e 

In [40]:
standings = pd.read_csv("data_raw/constructor_standings.csv") 
stg = standings.merge(races[["raceId", "year"]], on="raceId", how="left")

stg_ferrari = stg.query("constructorId == @ferrari_id and 1981 <= year <= 2020")

ferrari_season_points = (
    stg_ferrari.groupby("year", as_index=False)
               .agg(season_points=("points", "max"))
               .sort_values("year")
)

max_pts = ferrari_season_points["season_points"].max()
best_years = ferrari_season_points[ferrari_season_points["season_points"] == max_pts]
print(best_years)  

    year  season_points
37  2018          571.0
